# Pull all features for an area

In this notebook, I'm developing and testing code that pulls and standardizes all 
of the map features that we are interested in, for a box of a given size.

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape


## Parameters

In [ ]:
# AOIs are squares of this dimension, in meters.
aoi_size = 10000

# For testing
# lat, lon = 42.981163, -70.946524 # Exeter NH
# lat, lon = 43.077132, -70.757544 # Portsmouth NH
# lat, lon = 42.969680, -71.000339 # pickpocket
# lat, lon = 42.934, -72.278  # Keene NH
lat, lon = 42.996,-71.455  # Manchester NH


## Preliminaries

In [ ]:
# Define a local map projection

center_lat = lat
center_lon = lon
x0 = aoi_size / 2
y0 = aoi_size / 2

proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
+k=1.0 +x_0={x0} +y_0={y0} +datum=WGS84 +units=m +no_defs
"""

ltm_crs = pyproj.CRS.from_proj4(proj_def)
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform


In [ ]:
print(proj_forward(center_lon, center_lat))

In [ ]:
# Define a polygon for the AOI bounds.
x0, y0 = 0, 0
x1, y1 = aoi_size, aoi_size
xx = [x0, x1, x1, x0, x0]
yy = [y0, y0, y1, y1, y0]
bbox = shapely.Polygon(list(zip(xx, yy)))
print(bbox)

In [ ]:
# Use that projection to define lon/lat nbounds. Make sure the bounds go 
# a little farther out than necessary to avoid edge artifacts from map projections.
buf = 100

lon0, lat0 = proj_inverse(x0 - buf, y0 - buf)
lon1, lat1 = proj_inverse(x1 + buf, y1 + buf)
print(lon0, lat0, lon1, lat1)
print(proj_forward(lon0, lat0))
print(proj_forward(lon1, lat1))


In [ ]:
# Define the boundng box to be used to query OSM.
query_bounds = [lon0, lat0, lon1, lat1]
center_lon = (lon0 + lon1) / 2
center_lat = (lat0 + lat1) / 2

In [ ]:
tags = {
    'landuse': True,
    'place': True,
    'highway': True,
    'railway': True,
    'aeroway': True,
    'bridge': True,
    'tunnel': True,
    'power': True,
    'natural': True,
    'waterway': True,
    'landcover': True,
    'building': True,
    'amenity': True,
    'shop': True,
    'leisure': True
}
features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d features' % len(features))

In [ ]:
import gent

orecs = [] 

for feature in features.to_dict('records'):
    
    geomxy = shapely.ops.transform(proj_forward, feature['geometry'])
    geomxy = geomxy.intersection(bbox)
    if geomxy.is_empty:
        continue
    gtype = geomxy.geom_type

    for rule in gent.rules:
        if gtype == rule['gtype']:
            osm_key = rule['osm_key']
            if osm_key in feature:
                osm_value = str(feature[osm_key])
                if osm_value in rule['osm_values']:
                    orecs.append({
                        'category': rule['gent_category'],
                        'label': rule['gent_label'],
                        'geomxy': geomxy,
                        'gtype': gtype
                    })

print(len(orecs))

In [ ]:
df = pd.DataFrame(orecs)
print(len(df))

df2 = df.drop_duplicates()
print(len(df2))

In [ ]:
df2[['category', 'label', 'gtype']].value_counts().sort_index()

In [ ]:
import folium
from shapely.geometry import Polygon
import geopandas as gpd

# Define a list of Shapely Polygon objects
polygons = [
    shapely.ops.transform(proj_inverse, z['geomxy'])
    for z in list(filter(lambda x: x['label'] == 'residential', orecs))
]

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': polygons}, crs='EPSG:4326')

# Create a Folium map centered at the mean location
center = gdf.geometry.unary_union.centroid
m = folium.Map(location=[center.y, center.x], zoom_start=14)

# Add each polygon to the map
for poly in gdf.geometry:
    geo_json = folium.GeoJson(data=gpd.GeoSeries(poly).__geo_interface__)
    geo_json.add_to(m)

# Display the map in the notebook
m


In [ ]:
x0, y0 = None, None
x1, y1 = None, None

for rec in df2.to_dict('records'):
    g = rec['geomxy']
    b = g.bounds
    if x0 is None or b[0] < x0:
        x0 = b[0]
    if y0 is None or b[1] < y0:
        y0 = b[0]
    if x1 is None or b[2] > x1:
        x1 = b[0]
    if y1 is None or b[3] > y1:
        y1 = b[0]
        
x0, y0, x1, y1    